<a href="https://colab.research.google.com/github/chitalekunal/ai_governance/blob/main/bias_aware_credit_approval_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Bias-Aware Credit Approval System (EU AI Act Compliance)

In [1]:
# @title Step 1: The "Lazy" Model
# @markdown We will train a random forest on set of banking data. As per the understanding, the model will cheat based on the bias data.